<a href="https://colab.research.google.com/github/jerry71867/LLM/blob/main/16_gensim_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Word2Vec實作**

*   字詞所代表的意義非常多元，在不同狀況下，會代表不同意思。要把多元意思用單一向量表示，則必須要進行word embedding的動作，也就是把高維向量降為低維向量的過程

*   之前介紹過，利用分散式表示法來表達字詞向量，例如PMI、SVD..統計法..等
*   2013年神經網路盛行後，Tomas Mikolov利用神經網路訓練方式，來獲得字詞的表達向量，獲得很棒的成果。一般認為是利用神經網路模擬人類的理解能力，獲得不錯的分布空間所得到的成果。
*   本範例以維基百科wiki部分資料作範例
*   資料來源：https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
*   利用結巴分詞(jieba)進行斷詞，gensim套件進行word2vec計算
本範例約需1小時長時間執行







In [ ]:
!wget https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2

--2024-01-04 12:23:11--  https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230609687 (220M) [application/octet-stream]
Saving to: ‘zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2’

zhwiki-20231201-pag 100%[===================>] 219.93M  3.05MB/s    in 61s     

2024-01-04 12:24:12 (3.61 MB/s) - ‘zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2’ saved [230609687/230609687]



opencc是繁簡轉換工具

In [ ]:

!pip install opencc-python-reimplemented

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 3.3 MB/s eta 0:00:00


gensim是訓練word2vec的函式庫

In [ ]:

from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})

In [ ]:

wiki_corpus

In [ ]:

next(iter(wiki_corpus.get_texts()))[:10]

['歐幾里得',
 '西元前三世紀的古希臘數學家',
 '現在被認為是幾何之父',
 '此畫為拉斐爾的作品',
 '雅典學院',
 '数学',
 '是研究數量',
 '屬於形式科學的一種',
 '數學利用抽象化和邏輯推理',
 '從計數']

### 把wiki的資料檔案，轉換成連續文字的txt檔案

In [ ]:
text_num = 0
#初始化一個變數text_num，用於計算處理的文本數量。
with open('wiki_text.txt', 'w', encoding='utf-8') as f:
#打開一個檔案wiki_text.txt以進行寫入，使用 UTF-8 編碼。
    for text in wiki_corpus.get_texts():
    #遍歷 wiki_corpus 中的文本。
        f.write(' '.join(text)+'\n')
        #將處理過的文本以空格分隔的詞彙序列的形式寫入文件。
        text_num += 1
        #計算處理的文本數量。
        if text_num % 10000 == 0:
        #當處理的文本數量達到10000篇時，執行下面的代碼塊。
            print('{} articles processed.'.format(text_num))
            #列印目前處理的文本數量。

    print('{} articles processed.'.format(text_num))
    #在所有文本處理完成後，再次列印總處理的文本數量。


10000 articles processed.

20000 articles processed.

30000 articles processed.

32705 articles processed.

In [ ]:
import jieba
from opencc import OpenCC


# Initial
cc = OpenCC('s2t')
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
train_data = cc.convert(train_data)
train_data = jieba.lcut(train_data)
train_data = [word for word in train_data if word != '']
train_data = ' '.join(train_data)
open('seg.txt', 'w', encoding='utf-8').write(train_data)


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.966 seconds.
DEBUG:jieba:Loading model cost 0.966 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


28134217

In [ ]:
from gensim.models import word2vec


# Settings
seed = 666
sg = 0
window_size = 10
#vector_size = 100
min_count = 1
workers = 8
#epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('seg.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    #size=vector_size,
    workers=workers,
    #iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')


In [ ]:
from gensim.models import word2vec

string = '電腦'
model = word2vec.Word2Vec.load('word2vec.model')
print(string)

for item in model.wv.most_similar(string):
    print(item)

電腦
('計算機', 0.7747930288314819)

('軟體', 0.7732110619544983)

('程式', 0.7476573586463928)

('pda', 0.7414882779121399)

('遊戲機', 0.7409443855285645)

('硬體', 0.7400156259536743)

('pc', 0.7379648089408875)

('ibm', 0.728009045124054)

('家用', 0.7150958776473999)

('手機', 0.7146145701408386)